# Serve data from OpenAgua into WEAP using WaMDaM

#### By Adel M. Abdallah, Utah State University, December 2018

Execute the following cells by pressing `Shift-Enter`, or by pressing the play button <img style='display:inline;padding-bottom:15px' src='play-button.png'> on the toolbar above.



<a name="Import"></a>
# 1. Import python libraries 

In [1]:
# 1. Import python libraries 
### set the notebook mode to embed the figures within the cell

import sqlite3
import numpy as np
import pandas as pd
import getpass
from hs_restclient import HydroShare, HydroShareAuthBasic
import os

import plotly
plotly.__version__
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
offline.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)         # initiate notebook for offline plot

import os
import csv
from collections import OrderedDict
import sqlite3
import pandas as pd
import numpy as np
from IPython.display import display, Image, SVG, Math, YouTubeVideo
import urllib
import calendar

print 'The needed Python libraries have been imported'

The needed Python libraries have been imported


# 2. Connect to the WaMDaM SQLite on HydroSahre
### Provide the HydroShare ID for your resource
Example  
https://www.hydroshare.org/resource/8760f63a1f5545c380a3dd7ec847cb61/
 
resource_id='85e9fe85b08244198995558fe7d0e294'

In [2]:
username = 'amabdallah'
password = 'MyHydroShareWorld'

auth = HydroShareAuthBasic(username=username, password=password)

hs = HydroShare(auth=auth)

print 'Connected to HydroShare'

# Then we can run queries against it within this notebook :)  
resource_url='https://www.hydroshare.org/resource/123b2781b73742a18bfadd21124c6194/' 


resource_id= resource_url.split("https://www.hydroshare.org/resource/",1)[1] 
resource_id=resource_id.replace('/','')

print resource_id

resource_md = hs.getSystemMetadata(resource_id)
# print resource_md
print 'Resource title'
print(resource_md['resource_title'])
print '----------------------------'

resources=hs.resource(resource_id).files.all()

file = ""

for f in hs.resource(resource_id).files.all():

    file += f.decode('utf8')

import json

file_json = json.loads(file)

for f in file_json["results"]:

    FileURL= f["url"]
    
    SQLiteFileName=FileURL.split("contents/",1)[1] 

cwd = os.getcwd()


conn = sqlite3.connect(SQLiteFileName,timeout=10)

print 'Connected to the SQLite file= '+ SQLiteFileName
print 'done'

Connected to HydroShare
123b2781b73742a18bfadd21124c6194
Resource title
Final datasets
----------------------------
Connected to the SQLite file= WEAP_No_OA.sqlite
done


<a name="Connect"></a>
# 4. Connect to the WaMDaM populated SQLite file published in HydroShare to query its data and serve them to WEAP scenarios


In [16]:
# Then we can run queries against it within this notebook :)  

# the SQLite file is published here 
#https://github.com/WamdamProject/WaMDaM_UseCases/blob/master/UseCases_files/3SQLite_database/BearRiverDatasets_June_2018.sqlite

WaMDaM_SQLite_Name='WEAP.sqlite'
# WaMDaM_SQLite_Name='replicateWaMDaM.sqlite'

# Test if the connection works 
conn = sqlite3.connect(WaMDaM_SQLite_Name)
df = pd.read_sql_query("SELECT ResourceTypeAcronym   FROM ResourceTypes Limit 1 ", conn)
print df
                       
print '--------------------'                        
print '\n Connected to the WaMDaM SQLite file called'+': '+ WaMDaM_SQLite_Name

  ResourceTypeAcronym
0                WEAP
--------------------

 Connected to the WaMDaM SQLite file called: WEAP.sqlite


<a name="ConnectWEAP"></a>
# 2. Connect to the WEAP API

### You need to have WEAP already installed on your machine

First make sure to have a copy of the Water Evaluation And Planning" system (WEAP) installed on your local machine (Windows). If you don’t have it installed, download and install the WEAP software which allows you to run the Bear River WEAP model and its scenarios for Use Case 5. https://www.weap21.org/. You need to have a WEAP License. See here (https://www.weap21.org/index.asp?action=217). If you're interested to learning about WEAP API, check it out here: http://www.weap21.org/WebHelp/API.htm    


## Install dependency and register WEAP
### 2.1. Install pywin32 extensions which provide access to many of the Windows APIs from Python.
**Choose on option**
* a. Install using an executable basedon your python version. Use version for Python 2.7
https://github.com/mhammond/pywin32/releases 

**OR**   

* b. Install it using Anaconda terminal @ https://anaconda.org/anaconda/pywin32

Type this command in the Anaconda terminal as Administrator  

    conda install -c anaconda pywin32 


**OR**

* c. Install from source code (for advanced users) 
https://github.com/mhammond/pywin32


### 2.2. Register WEAP with Windows 


This use case only works on a local Jupyter Notebook server installed on your machine along with WEAP. So it does not work on the online Notebooks in Step 2.1. You need to install Jupyter Server in Step 2.2 then proceed here.




* **Register WEAP with Windows to allow the WEAP API to be accessed**    
Use Windows "Command Prompt". Right click and then <font color=red>**run as Administrator**</font>, navigate to the WEAP installation directory such as and then hit enter  

```
cd C:\Program Files (x86)\WEAP
```

Then type the following command in the command prompt and hit enter   
```
WEAP /regserver
```


<img src="https://github.com/WamdamProject/WaMDaM-software-ecosystem/blob/master/mkdocs/Edit_MD_Files/QuerySelect/images/RegisterWEAP_CMD.png?raw=true" style="float:center;width:700px;padding:20px"> 
Figure 1: Register WEAP API with windows using the Command Prompt (Run as Administrator)



# 3. Download the Bear River WEAP model and connect Jupyter Notebook to WEAP API

Clone or download all this GitHub repo   
https://github.com/WamdamProject/WaMDaM_UseCases  

In your local repo folder, go to the    
    
    C:\Users\Adel\Documents\GitHub\WaMDaM_UseCases/UseCases_files/1Original_Datasets_preperation_files/WEAP/Bear_River_WEAP_Model_2017

Copy this folder **Bear_River_WEAP_Model_2017** and paste it into **WEAP Areas** folder on your local machine. For example, it is at   

    C:\Users\Adel\Documents\WEAP Areas  


In [14]:
# this library is needed to connect to the WEAP API
import win32com.client

# this command will open the WEAP software (if closed) and get the last active model
# you could change the active area to another one inside WEAP or by passing it to the command here
#WEAP.ActiveArea = "BearRiverFeb2017_V10.9"


WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

# WEAP.Visible = 'FALSE'


print WEAP.ActiveArea.Name 
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Original"  
print WEAP.ActiveArea.Name 

WEAP.Areas("Bear_River_WEAP_Model_2017_Original").Open
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Original"  
print WEAP.ActiveArea.Name


print 'Connected to WEAP API and the '+ WEAP.ActiveArea.Name + ' Area'
print '-------------'
if not WEAP.Registered:
    print "Because WEAP is not registered, you cannot use the API"

# get the active WEAP Area (model) to serve data into it 

# ActiveArea=WEAP.ActiveArea.Name 


# get the active WEAP scenario to serve data into it 
print '-------------'

ActiveScenario= WEAP.ActiveScenario.Name
print '\n ActiveScenario= '+ActiveScenario
print '-------------'

WEAP_Area_dir=WEAP.AreasDirectory
print WEAP_Area_dir


print "\n \n You're connected to the WEAP API"

Bear_River_WEAP_Model_2017_Conservation
Bear_River_WEAP_Model_2017_Conservation
Bear_River_WEAP_Model_2017_Conservation
Connected to WEAP API and the Bear_River_WEAP_Model_2017_Conservation Area
-------------
-------------

 ActiveScenario= 
-------------
C:\Users\Adel\Documents\WEAP Areas\

 
 You're connected to the WEAP API


<a name="CreateWEAP_Area"></a>
# 3.1 Create a copy of the original WEAP Area to use while keeping the orignial as-as for any later use


<a name="AddScenarios"></a>
### Add a new CacheCountyUrbanWaterUse scenario from the Reference original WEAP Area:  

### You can always use this orignal one and delete any new copies you make afterwards.

In [15]:
# Create a copy of the WEAP AREA to serve the updated Hyrym Reservoir to it 


# Delete the Area if it exists and then add it. Start from fresh
Area="Bear_River_WEAP_Model_2017_Conservation"

if not WEAP.Areas.Exists(Area):
    WEAP.SaveAreaAs(Area)


WEAP.ActiveArea.Save
WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Conservation"  
print  'ActiveArea= '+WEAP.ActiveArea.Name

#  Add new Scenario
#  Add(NewScenarioName, ParentScenarioName or Index):   
#  Create a new scenario as a child of the parent scenario specified.    
#  The new scenario will become the selected scenario in the Data View.  
    
    
    
WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")
# WEAP.Visible = FALSE


WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Conservation"  

print  'ActiveArea= '+ WEAP.ActiveArea.Name

Scenario=[]
Scenario='CacheCountyUrbanWaterUse'

# Delete the scenario if it exists and then add it. Start from fresh
if WEAP.Scenarios.Exists(Scenario):
    WEAP.Scenarios(Scenario).Delete(True)
else:
    WEAP.Scenarios.Add(Scenario,'Reference')
    
WEAP.ActiveArea.Save
WEAP.SaveArea

WEAP.Quit

# or add the scenarios one by one using this command   
    
# Make a copy from the reference (base) scenario
# WEAP.Scenarios.Add('UpdateCacheDemand','Reference')
print '---------------------- \n'
print 'Scenarios added to the original WEAP area'    


WEAP.Quit

print 'Connection with WEAP API is disconnected'

ActiveArea= Bear_River_WEAP_Model_2017_Conservation
ActiveArea= Bear_River_WEAP_Model_2017_Conservation
---------------------- 

Scenarios added to the original WEAP area
Connection with WEAP API is disconnected


<a name="QuerySupplyDataLoadWEAP"></a>
# 4.1.A Query Cache County seasonal "Monthly Demand" for the three sites: Logan Potable, North Cache Potable, South Cache Potable

### The data comes from OpenAgua

In [29]:
# Use Case 3.1Identify_aggregate_TimeSeriesValues.csv
# plot aggregated to monthly and converted to acre-feet time series data of multiple sources


# Logan Potable
# North Cache Potable
# South Cache Potable

# 2.2Identify_aggregate_TimeSeriesValues.csv
Query_UseCase_URL="""
https://raw.githubusercontent.com/WamdamProject/WaMDaM_JupyterNotebooks/master/3_VisualizeShare/WEAP/Query_demand_sites.sql
"""

# Read the query text inside the URL
Query_UseCase_text = urllib.urlopen(Query_UseCase_URL).read()


# return query result in a pandas data frame
result_df_UseCase= pd.read_sql_query(Query_UseCase_text, conn)

# uncomment the below line to see the list of attributes
# display (result_df_UseCase)
seasons_dict = dict()

subsets = result_df_UseCase.groupby('InstanceName')
for subset in subsets.groups.keys():
    df_Seasonal = subsets.get_group(name=subset)
    df_Seasonal=df_Seasonal.reset_index()
    SeasonalParam = ''
    for i in range(len(df_Seasonal['SeasonName'])):
        m_data = df_Seasonal['SeasonName'][i]
        n_data = float(df_Seasonal['SeasonNumericValue'][i])*10
        SeasonalParam += '{},{}'.format(m_data, n_data)
        if i != len(df_Seasonal['SeasonName']) - 1:
            SeasonalParam += ','

    Seasonal_value="MonthlyValues("+SeasonalParam+")"
    seasons_dict[subset]=(Seasonal_value)   
print seasons_dict

print 'Query and data preperation are done'

{u'North Cache Potable': 'MonthlyValues(Oct,1352.57,Nov,735.715,Dec,301.5,Jan,301.5,Feb,735.72,Mar,1352.57,Apr,2668.99,May,3385.79,Jun,3747.638,Jul,3747.64,Aug,3385.79,Sep,2565.613)', u'Logan Potable': 'MonthlyValues(Oct,6719.475,Nov,4651.256,Dec,3195.416,Jan,3195.416,Feb,4651.26,Mar,6719.475,Apr,11133.21,May,13536.51,Jun,14749.71,Jul,14749.71,Aug,13536.51,Sep,10786.59)', u'South Cache Potable': 'MonthlyValues(Oct,10870.66,Nov,6832.357,Dec,3989.75,Jan,3989.75,Feb,6832.36,Mar,10870.66,Apr,19488.72,May,24181.28,Jun,26550.12,Jul,26550.12,Aug,24181.28,Sep,18811.91)'}
Query and data preperation are done


<a name="LoadFlow"></a>
# 4.1.B Load the seasonal demand data with conservation  into WEAP

In [30]:
# 9. Load the seasonal data into WEAP
WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")
# WEAP.Visible = FALSE


print WEAP.ActiveArea.Name

WEAP.ActiveScenario = "CacheCountyUrbanWaterUse"
print WEAP.ActiveScenario.Name


DemandSites=['Logan Potable','North Cache Potable','South Cache Potable']

AttributeName='Monthly Demand'

# Get the Instance Name and Attribute names and pass them to 
# the function below to load their values into WEAP

# Seasonal_value contains the string of month value pairs 

for Branch in WEAP.Branches:
    for InstanceName in seasons_dict.keys():
        if Branch.Name == InstanceName:
            GetInstanceFullBranch = Branch.FullName
            WEAP.Branch(GetInstanceFullBranch).Variable(AttributeName).Expression = seasons_dict[InstanceName]

print '\n The seasonal dry headflow data have been sucsesfully loaded into WEAP'

WEAP.SaveArea

print '\n \n The updated data have been saved'

Bear_River_WEAP_Model_2017_Conservation
CacheCountyUrbanWaterUse

 The seasonal dry headflow data have been sucsesfully loaded into WEAP

 
 The updated data have been saved


# 5. Run WEAP and report back the unment demand at the Bird Refuge demand site

<font color=green>**Please wait, it will take ~3-7 minutes** to finish calcualting the two WEAP Areas with their many scenarios</font>

In [31]:
# Run WEAP

WEAP.Areas("Bear_River_WEAP_Model_2017_Conservation").Open
print WEAP.ActiveArea.Name

WEAP.ActiveArea = "Bear_River_WEAP_Model_2017_Conservation"  
print WEAP.ActiveArea.Name

WEAP.Calculate(2006,10,True)
WEAP.SaveArea

print '\n \n The calculation has been done and saved'
print WEAP.CalculationTime

UnmetDemandEstimate={}
Scenarios=[]
Scenarios=['Reference','CacheCountyUrbanWaterUse']
DemandSites=['Logan Potable','North Cache Potable','South Cache Potable']
for scen in Scenarios:
        print '\n-----------------------'
        print 'scenario='+ scen
        for year in range (1966,2006):
#         print 'Total Unmet Demand in 2002 for the Bird Refuge in Acre-feet='
            UnmetDemandEstimate[scen,year,'Logan Potable']=WEAP.ResultValue("\Demand Sites\Logan Potable: Unmet Demand[Acre-Foot]", year, 1, scen, year, WEAP.NumTimeSteps) 
            UnmetDemandEstimate[scen,year,'North Cache Potable']=WEAP.ResultValue("\Demand Sites\North Cache Potable: Unmet Demand[Acre-Foot]", year, 1, scen, year, WEAP.NumTimeSteps) 
            UnmetDemandEstimate[scen,year,'South Cache Potable']=WEAP.ResultValue("\Demand Sites\South Cache Potable: Unmet Demand[Acre-Foot]", year, 1, scen, year, WEAP.NumTimeSteps) 


# print UnmetDemandEstimate
WEAP.SaveArea
WEAP.Quit        
WEAP.SaveArea
keys = UnmetDemandEstimate.keys()
vals = UnmetDemandEstimate.values()
keys,values = zip(*UnmetDemandEstimate.items())




Bear_River_WEAP_Model_2017_Conservation
Bear_River_WEAP_Model_2017_Conservation

 
 The calculation has been done and saved
115

-----------------------
scenario=Reference

-----------------------
scenario=CacheCountyUrbanWaterUse


# 5.1 post process the result 

In [34]:
WEAPArea=['Bear_River_WEAP_Model_2017_Conservation']
Scenarios=['Reference','CacheCountyUrbanWaterUse']

UnmetDemandEstimate = pd.DataFrame(columns = ['Reference','CacheCountyUrbanWaterUse'])
  
for scen in Scenarios:
    for year in range (1966,2006):
#         print '\n-----------------------'
#         print 'scenario='+ scen
#         print 'Total Unmet Demand in 2002 for the Bird Refuge in Acre-feet='

        value=WEAP.ResultValue("\Demand Sites\Bird Refuge: Unmet Demand[Acre-Foot]", year, 1, scen, year, WEAP.NumTimeSteps) 
        if scen=='Reference':
            UnmetDemandEstimate.loc[year, ['Reference']]=value
        else:
            scen=='CacheCountyUrbanWaterUse'
            UnmetDemandEstimate.loc[year, ['CacheCountyUrbanWaterUse']]=value



            
# find annual demand at the Bird Refuge
for Branch in WEAP.Branches:
    for V in Branch.Variables:
        if Branch.Name == 'Bird Refuge' and V.Name== 'Monthly Demand':
            GetExpresValue = V.Expression

print GetExpresValue
ExpresValue1 = GetExpresValue.split('MonthlyValues(')[-1].split(')')[0].split(',')
# print ExpresValue1
it = iter(ExpresValue1)     
Demand=dict(zip(it, it))
# print Demand
Annual_Demand=0

for key in Demand:
    x= Demand[key] 
    Annual_Demand=Annual_Demand+float(x)
print 'Annual_Demand='+ str(Annual_Demand)

MonthlyValues( Oct, 42150,  Nov, 3406,  Dec, 0,  Jan, 0,  Feb, 4258,  Mar, 60884,  Apr, 59181,  May, 61309,  Jun, 46834,  Jul, 50240,  Aug, 43002,  Sep, 54497 )
Annual_Demand=425761.0


[1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005]


# 6. Plot the unmet demad for all the scenarios  and years


In [35]:
import numpy

import plotly.plotly as py
import plotly.graph_objs as go
# xx=['Reference','Cache County <br> demand','Hyrum  <br> Bathymetry','Headflow <br> supply']
# print UnmetDemandEstimate[['Reference','year']]

Reference_vals =UnmetDemandEstimate['Reference'].tolist()
CacheCountyUrbanWaterUse_vals =UnmetDemandEstimate['CacheCountyUrbanWaterUse'].tolist()

# print Reference_vals


Reference_vals_perc =(numpy.array([Reference_vals]))/Annual_Demand*100
CacheCountyUrbanWaterUse_vals_perc =(numpy.array([CacheCountyUrbanWaterUse_vals]))/Annual_Demand*100

    

# print UpdateBearHeadFlowDry_vals_perc.tolist()


UnmetDemandEstimate['Reference']

trace1 = go.Box(

    y=Reference_vals_perc[0],
    name = 'Reference',
    marker = dict(
        color = '#BFB2FF',
    
    ),boxmean=True
)


trace2 = go.Box(

    y=CacheCountyUrbanWaterUse_vals_perc[0],
    name = ' CacheCountyUrbanWaterUse',
    marker = dict(
        color = '#260F99'
    ),boxmean=True
)

layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Annual unmet demand (%)",
        tickformat= ',',
#         showline=True,
        dtick='5',
        ticks='outside',
        ticklen=10,
         range = ['0', '15']


                ),
    
    xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
        ticks='outside',
         tickfont=dict(size=22),
        ticklen=10
                    ),
    legend=dict(
        x=0.1,y=0.6,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
    width=1100,
    height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=130,b=200),
    font=dict(size=25,family='arial',color='#00000f'),
    showlegend=False
)
data = [trace1, trace2]


# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       

print "Figure 9 is replicated!!"

Figure 9 is replicated!!


# Plot 2

<a name="Close"></a>
# 7. Close the SQLite and WEAP API connections

In [66]:
import numpy

import plotly.plotly as py
import plotly.graph_objs as go
# xx=['Reference','Cache County <br> demand','Hyrum  <br> Bathymetry','Headflow <br> supply']
# print UnmetDemandEstimate[['Reference','year']]

Reference_vals =UnmetDemandEstimate['Reference'].tolist()
CacheCountyUrbanWaterUse_vals =UnmetDemandEstimate['CacheCountyUrbanWaterUse'].tolist()
Years=UnmetDemandEstimate.index.tolist()

UnmetDemandEstimate['Reference']

trace1 = go.Scatter(

    x=Years,
    y=Reference_vals,
    mode = 'lines+markers',
    name = 'Reference',
    marker = dict(
        color = '#BFB2FF',
    
    )
)


trace2 = go.Scatter(

    x=Years,
    y=CacheCountyUrbanWaterUse_vals,
    name = ' CacheCountyUrbanWaterUse',
    mode = 'lines+markers',
    marker = dict(
        color = '#260F99'
    )
)

layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Annual unmet demand (%)",
        tickformat= ',',
#         showline=True,
#         dtick='5',
#         ticks='outside',
#         ticklen=10,
#          range = ['0', '15']


                ),
    
    xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
        ticks='outside',
#          tickfont=dict(size=22),
#         ticklen=10
                    ),
    legend=dict(
        x=0.1,y=0.6,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
#     width=1100,
#     height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=130,b=200),
    font=dict(size=25,family='arial',color='#00000f'),
    showlegend=False
)
data = [trace1, trace2]


# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       

print "Figure 9 is replicated!!"

Figure 9 is replicated!!


# plot 3

In [69]:
import numpy

import plotly.plotly as py
import plotly.graph_objs as go
# xx=['Reference','Cache County <br> demand','Hyrum  <br> Bathymetry','Headflow <br> supply']
# print UnmetDemandEstimate[['Reference','year']]

Reference_vals =UnmetDemandEstimate['Reference'].tolist()
CacheCountyUrbanWaterUse_vals =UnmetDemandEstimate['CacheCountyUrbanWaterUse'].tolist()
Years=UnmetDemandEstimate.index.tolist()

UnmetDemandEstimate['Reference']

trace1 = go.Scatter(

    x=CacheCountyUrbanWaterUse_vals,
    y=Reference_vals,
    mode = 'markers',
    name = 'Reference',
    marker = dict(
        color = '#BFB2FF',
    
    )
)


# trace2 = go.Scatter(

#     x=Years,
#     y=CacheCountyUrbanWaterUse_vals,
#     name = ' CacheCountyUrbanWaterUse',
#     mode = 'lines+markers',
#     marker = dict(
#         color = '#260F99'
#     )
# )

layout = dict(
    #title = "Use Case 3.3",
    yaxis = dict(
        title = "Annual unmet demand (%)",
        tickformat= ',',
#         showline=True,
#         dtick='5',
#         ticks='outside',
#         ticklen=10,
#          range = ['0', '15']


                ),
    
    xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
        ticks='outside',
#          tickfont=dict(size=22),
#         ticklen=10
                    ),
    legend=dict(
        x=0.1,y=0.6,
          bordercolor='#00000f',
            borderwidth=2
        
               ),
#     width=1100,
#     height=700,
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    margin=go.Margin(l=130,b=200),
    font=dict(size=25,family='arial',color='#00000f'),
    showlegend=False
)
# data = [trace1, trace2]

data = [trace1]

# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'jupyter/UnmentDemand@BirdRefuge' )       

print "Figure 9 is replicated!!"

Figure 9 is replicated!!


In [36]:
# 9. Close the SQLite and WEAP API connections
conn.close()

print 'connection disconnected'

# Uncomment 
# WEAP.SaveArea

# Or 
# NewWEAPCopyName=ActiveArea+"Test"
# print NewWEAPCopyName

# Call API function to save WEAP
# WEAP.SaveAreaAS(NewWEAPCopyName)

# this command will close WEAP
# WEAP.Quit

print 'Connection with WEAP API is disconnected'

connection disconnected
Connection with WEAP API is disconnected


# The End :) Congratulations!